<a href="https://colab.research.google.com/github/Pestrdan/netology_research/blob/main/Deep_learning_(Pytorch)/DLL_35_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**1. Загрузите и подготовьте данные**

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import time
import numpy as np
from sklearn.preprocessing import StandardScaler

In [ ]:
df = pd.read_csv('housing.csv')

In [ ]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [ ]:
df.replace([np.inf, -np.inf], np.nan)  # Replace inf values with NaNs
df.dropna(inplace=True)  # Drop NaN values from the dataframe

**2.Разделите данные на test и train**

In [ ]:
#pre_pre_X = df.drop(['median_house_value', 'ocean_proximity'], axis=1)
#pre_X = torch.tensor(pre_pre_X.values, dtype = torch.float32)
#X = torch.nn.functional.normalize(pre_X)

In [ ]:
X_pd = df.drop(['median_house_value', 'ocean_proximity'], axis=1)

scaler = StandardScaler()
X_sc = pd.DataFrame(scaler.fit_transform(X_pd), columns=X_pd.columns)
X = torch.tensor(X_sc.values, dtype = torch.float32)

In [ ]:
y = torch.tensor(df['median_house_value'].values, dtype = torch.float32).reshape(-1, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

In [ ]:
BATCH_SIZE=32

In [ ]:
train = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE)
test = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE)

**3. Создать модель (объект) класса. Для создания объекта можно использовать класс Sequential**

In [ ]:
model = torch.nn.Sequential(torch.nn.Linear(8, 1))

In [ ]:
loss = torch.nn.MSELoss()

In [ ]:
trainer = torch.optim.SGD(model.parameters(), lr=0.0008)

**4. Обучите модель на train данных**

In [ ]:
epochs = 100

In [ ]:
for epoch in range(epochs):
  model.train()
  for X, y in train:
    trainer.zero_grad()
    y_pred = model(X)
    l = loss(y_pred, y)
    l.backward()
    trainer.step()
  if (epoch + 1) % 10 == 0:
    print(f'Эпоха [{epoch + 1}/{epochs}], Loss: {l.item():.4f}')

Эпоха [10/100], Loss: 2779790848.0000
Эпоха [20/100], Loss: 2628220928.0000
Эпоха [30/100], Loss: 2594249472.0000
Эпоха [40/100], Loss: 2591777024.0000
Эпоха [50/100], Loss: 2596712960.0000
Эпоха [60/100], Loss: 2602519296.0000
Эпоха [70/100], Loss: 2607630592.0000
Эпоха [80/100], Loss: 2611815168.0000
Эпоха [90/100], Loss: 2615185664.0000
Эпоха [100/100], Loss: 2617908992.0000


In [ ]:
with torch.no_grad():
  predicted = model(X_test).detach().numpy()

In [ ]:
X_test

tensor([[ 1.1633, -1.3309, -0.6856,  ..., -0.7942, -0.6420, -0.3619],
        [-0.8581,  1.1126, -2.1152,  ...,  1.6785,  1.1969,  0.3243],
        [ 0.7141, -0.8020,  0.1880,  ...,  0.7837,  0.7888, -0.3409],
        ...,
        [ 0.2699, -0.1466, -1.3210,  ..., -0.1156,  0.0616, -0.6917],
        [ 1.0735, -0.7271, -0.0503,  ..., -0.0997, -0.1450, -0.2036],
        [ 1.2332, -1.4339, -1.1621,  ...,  1.9741,  1.8142, -0.8139]])

In [ ]:
def calculate_mse(predicted, actual):

    assert len(predicted) == len(actual), "Длины массивов должны быть одинаковыми."

    # Вычисление суммы квадратов разностей
    squared_errors = np.square(predicted - actual)
    sum_squared_errors = np.sum(squared_errors)

    # Вычисление MSE
    mse = sum_squared_errors / len(predicted)

    return mse

In [ ]:
mse = calculate_mse(predicted, y_test.numpy())
print("TEST MSE:", mse)

TEST MSE: 4920150525.61977
